In [1]:
import urllib
from bs4 import BeautifulSoup

In [3]:
def buildTourneyYear(document, year):
    """
    Passes in a BeautifulSoup document of html.  Parses document, and returns a list of all games.
    """
    
    text = document.get_text().split('\n')

    #Computes index for full data list
    if year > 2010:
        first = text.index('SECOND ROUND')
        second = text.index('THIRD ROUND')
    else:
        first = text.index('FIRST ROUND')
        second = text.index('SECOND ROUND')
    if year == 2002:
        last = text.index('CHAMPIONSHIP') + 6
    else:
        last = text.index('NATIONAL CHAMPIONSHIP') + 6
    full = text[first:last]

    #Converts unicode elements to string
    full = [str(el) for el in full]

    #Removes empty strings from elements
    for num in range(full.count('')):
        full.remove('') 
    for num in range(full.count(' ')):
        full.remove(' ')
        
    return full

def buildRounds(gameList, gameRound, year):
    
    #Finds the index to separate first round and others
    if year > 2010:
        second = gameList.index('THIRD ROUND')
    else:
        second = gameList.index('SECOND ROUND')
    
    #Checks to see with round to build
    if gameRound == 'first':
        first_round = gameList[2:second]
        [first_round.remove(game) for game in first_round if game[:3] != 'No.']
        return first_round
    else:
        other_rounds = gameList[second + 2:]
        [other_rounds.remove(game) for game in other_rounds if ',' not in game]

        return other_rounds

def buildYearWinners(first, other):
    """
    Builds a dictionary of seeds as keys and a list of winning teams of those seeds.
    first = list
    other = list
    returns a dictionary
    """
    
    #Builds a dictionary with empty lists of values for each seed
    year_winners = {}
    for j in range(1,17):
        year_winners[j] = []

    #Adds winning seeded teams for first round
    for game in first:
        seed = int(game[4:6].rstrip())
        if seed < 10:
            winner = game.split(',')[0][6:-3].rstrip()
        else:
            winner = game.split(',')[0][7:-3].rstrip()
        year_winners[seed].append(winner)

    #Adds winning seeded teams for remaining rounds
    for game in other:
        winner = game.split(',')[0][:-3].lstrip()
        team_lst = year_winners.values()
        for lst in team_lst:
            if winner in lst:
                lst.append(winner)
    
    return year_winners

def tourneyWinners(beg, end):
    
    winners = []

    years = range(beg,end)

    for i in range(len(years)):

            html = urllib.urlopen('http://www.cbssports.com/collegebasketball/ncaa-tournament/history/yearbyyear/' + str(years[i]))
            document = BeautifulSoup(html)

            try:
                full = buildTourneyYear(document, years[i])
                first_round = buildRounds(full,'first', years[i])
                other_rounds = buildRounds(full,'other', years[i])
            except UnicodeEncodeError:
                print 'error in year ' + str(years[i])

            try:
                year_winners = buildYearWinners(first_round, other_rounds)
            except ValueError:
                print 'error in year ' + str(years[i])
            winners.append(year_winners)
    
    return winners

In [7]:
#2010
html = urllib.urlopen('http://www.cbssports.com/collegebasketball/ncaa-tournament/history/yearbyyear/2010')
document = BeautifulSoup(html)
text = document.get_text().split('\n')

#Computes index for full data list
first = text.index('FIRST ROUND')
second = text.index('THIRD ROUND')
last = text.index('NATIONAL CHAMPIONSHIP') + 6
full = text[first:last]
full.remove(full[97])

#Converts unicode elements to string
full = [str(el) for el in full]

#Removes empty strings from elements
for num in range(full.count('')):
    full.remove('') 
for num in range(full.count(' ')):
    full.remove(' ')

ValueError: 'THIRD ROUND' is not in list

In [10]:
second = full.index('SECOND ROUND')
    
first_round = full[3:second]
[first_round.remove(game) for game in first_round if game[:3] != 'No.']
#print first_round
other_rounds = full[second + 2:]
[other_rounds.remove(game) for game in other_rounds if ',' not in game]
#print other_rounds

single_winner = {}
for j in range(1,17):
    single_winner[j] = []

    #Adds winning seeded teams for first round
for game in first_round:
    seed = int(game[4:6].rstrip())
    if seed < 10:
        winner = game.split(',')[0][6:-3].rstrip()
    else:
        winner = game.split(',')[0][7:-3].rstrip()
    single_winner[seed].append(winner)

    #Adds winning seeded teams for remaining rounds
for game in other_rounds:
    winner = game.split(',')[0][:-3].lstrip()
    team_lst = single_winner.values()
    for lst in team_lst:
        if winner in lst:
            lst.append(winner)
#single_winner[3].append('Baylor')    
print single_winner

{1: ['Syracuse', 'Kentucky', 'Duke', 'Syracuse', 'Kentucky', 'Duke', 'Kentucky', 'Duke', 'Duke', 'Duke', 'Duke'], 2: ['Ohio State', 'Kansas State', 'West Virginia', 'Villanova', 'Ohio State', 'Kansas State', 'West Virginia', 'West Virginia', 'West Virginia'], 3: ['Pittsburgh', 'New Mexico', 'Baylor', 'Baylor', 'Baylor'], 4: ['Maryland', 'Wisconsin', 'Purdue', 'Purdue'], 5: ['Michigan State', 'Butler', 'Texas A&M', 'Michigan State', 'Butler', 'Michigan State', 'Butler', 'Michigan State', 'Butler', 'Butler'], 6: ['Tennessee', 'Xavier', 'Tennessee', 'Xavier', 'Tennessee'], 7: ['BYU'], 8: ['Gonzaga', 'California'], 9: ['Northern Iowa', 'Wake Forest', 'Northern Iowa'], 10: ['Georgia Tech', 'Missouri', "St. Mary's"], 11: ['Washington', 'Old Dominion', 'Washington'], 12: ['Cornell', 'Cornell'], 13: ['Murray State'], 14: ['Ohio'], 15: [], 16: []}


In [4]:
first_winners = tourneyWinners(1985,2001)
second_winners = tourneyWinners(2002,2010)
third_winners = tourneyWinners(2011,2013)

In [5]:
final = []
final.extend(first_winners)
final.extend(second_winners)
final.extend(third_winners)

In [17]:
final.extend([single_winner])

In [86]:
totals = {}
for i in range(1,17):
    totals[i] = 0
    for year in final:
       totals[i] += len(year[i]) 
print totals

{1: 336, 2: 245, 3: 184, 4: 151, 5: 119, 6: 123, 7: 84, 8: 67, 9: 61, 10: 66, 11: 50, 12: 53, 13: 26, 14: 18, 15: 4, 16: 0}


In [6]:
#one_year = final[-1]
#for k, v in one_year.iteritems():
#    print k, v
print final

[{1: ["St. John's", 'Georgetown', 'Oklahoma', 'Michigan', "St. John's", 'Georgetown', 'Oklahoma', "St. John's", 'Georgetown', 'Oklahoma', "St. John's", 'Georgetown', 'Georgetown'], 2: ['Virginia Commonwealth', 'Georgia Tech', 'Memphis State', 'North Carolina', 'Georgia Tech', 'Memphis State', 'North Carolina', 'Georgia Tech', 'Memphis State', 'North Carolina', 'Memphis State'], 3: ['North Carolina State', 'Illinois', 'Duke', 'Kansas', 'North Carolina State', 'Illinois', 'North Carolina State'], 4: ['UNLV', 'Loyola (Ill.)', 'Ohio State', 'Loyola (Ill.)'], 5: ['Southern Methodist', 'Louisiana Tech', 'Maryland', 'Louisiana Tech', 'Maryland'], 6: ['Georgia'], 7: ['Alabama', 'Syracuse', 'Ala.-Birmingham', 'Notre Dame', 'Alabama'], 8: ['Temple', 'Villanova', 'Villanova', 'Villanova', 'Villanova', 'Villanova', 'Villanova'], 9: ['Arkansas', 'Illinois State'], 10: [], 11: ['Texas-El Paso', 'Boston College', 'Auburn', 'Boston College', 'Auburn'], 12: ['Kentucky', 'Kentucky'], 13: ['Navy'], 14: [

In [100]:
print len(one_year[4]) * 3 + len(one_year[2]) * 2 + one_year[3].count('Connecticut') * 2 + one_year[3].count('Purdue') * 2

53


In [101]:
print len(one_year[1]) * 1 + len(one_year[2]) * 2 + one_year[8].count('Butler') * 5 + one_year[11].count('Gonzaga') * 6

57


In [81]:
means_two = []

In [82]:
totals = []
for i in range(1,17):
    totals.append([])
    for year in final:
        totals[i-1].append(len(year[i]))

In [83]:
for seed in totals:
    means_two.append(sum(seed) / float(len(seed)))

In [85]:
for i in range(1,17):
    print i,means_two[i-1]

1 11.7586206897
2 8.79310344828
3 6.65517241379
4 5.55172413793
5 4.24137931034
6 4.24137931034
7 3.03448275862
8 2.65517241379
9 2.03448275862
10 2.34482758621
11 1.93103448276
12 1.86206896552
13 0.931034482759
14 0.620689655172
15 0.172413793103
16 0.0
